In [1]:
from utils import *

In [2]:
def haversine_distance(lat1, lon1, lat2, lon2):

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Radius of Earth in kilometers
    r = 6371.0
    return r * c

In [3]:
data = pd.read_pickle("./data/df_gesamt_15_08_prepocessed_einworner_added.pkl")

In [4]:
c_data = data[["Qid", "GJ", "Laenge", "Breite"]].copy()

In [5]:
c_data_2018 = c_data[c_data.GJ == 2018].copy()

In [6]:
# The parameters should be adjusted manually.
breite_bin_count = 2200
laenge_bin_count = 2200
c_data_2018["Breite_bin"] = pd.cut(c_data_2018.Breite, bins=np.linspace(c_data_2018.Breite.min(), c_data_2018.Breite.max(), breite_bin_count), labels=np.arange(breite_bin_count-1), precision=24, include_lowest=True)
c_data_2018["Laenge_bin"] = pd.cut(c_data_2018.Laenge, bins=np.linspace(c_data_2018.Laenge.min(), c_data_2018.Laenge.max(), laenge_bin_count), labels=np.arange(laenge_bin_count-1), precision=24, include_lowest=True)

In [7]:
# Check wheter the minimum cell is big enough
B = np.linspace(c_data_2018.Breite.min(), c_data_2018.Breite.max(), breite_bin_count)
L = np.linspace(c_data_2018.Laenge.min(), c_data_2018.Laenge.max(), laenge_bin_count)

a = haversine_distance(B[0], L[0], B[1], L[1])
b = haversine_distance(B[0], L[-2], B[1], L[-1]) 
c = haversine_distance(B[-2], L[-2], B[-1], L[-1])
d = haversine_distance(B[-2], L[0], B[-1], L[1])

print(f"{a:.4f},\n{b:.4f},\n{c:.4f},\n{d:.4f}")

0.4917,
0.4917,
0.4633,
0.4633


In [8]:
# Sliding window size: KxK kernel, stride is 1
K = 3
breite_windows = np.arange(K) + np.arange(breite_bin_count-K+1)[..., None]
laenge_windows = np.arange(K) + np.arange(laenge_bin_count-K+1)[..., None]

In [9]:
# Set the Qid column as index in order to get them from groups in future
c_data_2018.set_index("Qid", drop=True, inplace=True)

In [10]:
# Group the data by the corresponding bins
grouped_data_2018 = c_data_2018.groupby(["Breite_bin", "Laenge_bin"])

In [11]:
pairwise_qids = []

for idx, breite_window in enumerate(tqdm(breite_windows)):       
    for jdx, laenge_window in enumerate(laenge_windows):
        
        local_group = []
        
        # Combine all cells of KxK window
        for i in range(K):
            for j in range(K):
                one_cell_qids = grouped_data_2018.groups.get((breite_window[i], laenge_window[j]))
                
                # Check if the cell is not empty
                if one_cell_qids is not None:
                    local_group.append(one_cell_qids)
        
        # Check for empty window
        if len(local_group) == 0:
            continue
        
        # Concatenate the results for the window
        local_group = np.concatenate(local_group)
        
        # Repeat Qids (pairwise)
        pairwise_qid = np.stack([local_group.repeat(local_group.shape[0]), local_group[..., None].repeat(local_group.shape[0], 1).T.flatten()], axis=-1)

        pairwise_qids.append(pairwise_qid)

100%|████████████████████████████████████████████████████████████████| 2198/2198 [00:22<00:00, 99.24it/s]


In [12]:
# Concatenate all arrays
pairwise_qids = np.concatenate(pairwise_qids, axis=0)

In [13]:
# Make dataframe from the array
distances_2018 = pd.DataFrame(data=pairwise_qids, columns=["Qid_1", "Qid_2"])

In [14]:
# Add coordinates
distances_2018[["Laenge_1", "Breite_1"]] = c_data_2018.loc[pairwise_qids[:, 0]][["Laenge", "Breite"]].reset_index(drop=True)
distances_2018[["Laenge_2", "Breite_2"]] = c_data_2018.loc[pairwise_qids[:, 1]][["Laenge", "Breite"]].reset_index(drop=True)

In [15]:
# Calculate distances
distances_2018["distance"] = haversine_distance(distances_2018.Breite_1, distances_2018.Laenge_1,
                                                distances_2018.Breite_2, distances_2018.Laenge_2)

In [16]:
# Filter the data by the threshold
DISTANCE_THRESHOLD = 0.5
distances_2018 = distances_2018[distances_2018.distance <= DISTANCE_THRESHOLD]

In [17]:
# Set (Qid_1, Qid_2) paris as index and filter out duplicated results (because of the sliding window method)
distances_2018.set_index(["Qid_1", "Qid_2"], drop=True, inplace=True)
distances_2018 = distances_2018[~distances_2018.index.duplicated()]

In [18]:
distances_2018

Laenge_1   Breite_1   Laenge_2   Breite_2  distance
Qid_1    Qid_2                                                         
734001.0 734001.0  10.176197  47.355011  10.176197  47.355011  0.000000
         734002.0  10.176197  47.355011  10.176208  47.355008  0.000892
734002.0 734001.0  10.176208  47.355008  10.176197  47.355011  0.000892
         734002.0  10.176208  47.355008  10.176208  47.355008  0.000000
734003.0 734003.0  10.198566  47.365186  10.198566  47.365186  0.000000
...                      ...        ...        ...        ...       ...
719060.0 786860.0   8.908550  54.874788   8.909086  54.874755  0.034507
786860.0 417648.0   8.909086  54.874755   8.906080  54.871213  0.438323
         189223.0   8.909086  54.874755   8.908775  54.874165  0.068545
         719060.0   8.909086  54.874755   8.908550  54.874788  0.034507
         786860.0   8.909086  54.874755   8.909086  54.874755  0.000000

[2161364 rows x 5 columns]

In [19]:
# Save the result
# distances_2018.to_csv("./data/distances/distances_2018_new_algo.csv")